In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import graph_tool.all as gt
import networkx as nx
import os
import cowsay
sns.set_theme(style="whitegrid")
pwd = os.getcwd()
my_dpi=300
colmex = pd.read_csv(pwd + "/1_colmex_limpio.csv")
red_colmex = pd.read_csv(pwd +"/2_red_colmex_completa.csv")

In [2]:
# Una función que nos da el índice en Colmex al escribir el usuario
def index_colmex(username):
    try:
        index = colmex['username'].loc[lambda x: x==username].index[0]
        print("El index en Colmex de " + username + " es " + str(index))
    except IndexError:
        print ("El usuario no está en Colmex")

# Función que nos da el usuario y ID en Colmex al escribir el índice
def username_colmex(index):
    try:
        id = colmex['id'].iloc[index]
        user = colmex['username'].iloc[index]
        print("El username en Colmex del índice " + str(index) + " es " + user + ", ID: " + str(id))
    except IndexError:
        print ("El índice no está en Colmex")
# Función que nos dice cuántos following tiene un usuario
def following(username):
    summary = red_colmex.groupby('username').agg(following=('target', 'size')).reset_index().sort_values(['following'], ascending=False)
    return summary.loc[summary['username'] == username]

In [3]:
index_colmex('tomlibrarian')

El index en Colmex de tomlibrarian es 180


In [4]:
username_colmex(180)

El username en Colmex del índice 180 es tomlibrarian, ID: 113514502


## Red interna del Colmex

In [5]:
red_colmex = red_colmex.rename(columns={'id':'source'})
red_interna = red_colmex.loc[red_colmex['t_username'].isin(colmex.username)]

In [6]:
edges_interna_dir = red_interna[['source', 'target']].reset_index(drop=True)  # Creamos red dirigida (dir)
# Tenemos la red, en donde nos interesa conocer la dirección de los nodos para saber quienes son las personas a las que más siguen, los nodos objetivos.
edges_interna_dir.shape

(11184, 2)

In [7]:
# Prueba Followback
edges_interna_dir.loc[(edges_interna_dir['source'] == 118659352) & (edges_interna_dir['target'] == 260274699)]

,source,target
3898,118659352,260274699


In [8]:
# Prueba Followback
edges_interna_dir.loc[(edges_interna_dir['source'] == 260274699) & (edges_interna_dir['target'] == 118659352)]

,source,target
6898,260274699,118659352


In [ ]:
G = nx.DiGraph() # Nueva gráfica dirigida con Nx
G.add_edges_from(edges_interna_dir.values) # Utilizar los nodos de nuestro edgelist
G

In [ ]:
print("Number of nodes/usuarios", len(G))
print("Number of edges/directed follows", len(G.edges))

## Degree distribution

In [ ]:
degree = pd.DataFrame(G.degree, columns=['id', 'degree']).sort_values('degree', ascending=False)
degree = pd.merge(degree, colmex, how='left', on="id")
degree.head(10)

In [ ]:
sns.displot(degree['degree']).set(xlabel="Degree", ylabel = "Frecuencia")
plt.title('Degree distribution de la red dirigida Colmex.')
plt.savefig(pwd + '/figures/' + 'degree_distribution_interna_dir.png', dpi=my_dpi,bbox_inches='tight')

## Assortativity

In [ ]:
nodes = degree.dropna()

In [ ]:
att_status = dict(zip(nodes['id'], nodes['status']))
att_center = dict(zip(nodes['id'], nodes['center']))

In [ ]:
nx.set_node_attributes(G, att_status, 'status')
nx.set_node_attributes(G, att_center, 'center')

In [ ]:
# G.nodes(data=True)

In [ ]:
nx.attribute_assortativity_coefficient(G, 'status')

In [ ]:
nx.attribute_assortativity_coefficient(G, 'center')

## Clustering

In [ ]:
# Indivdual
clust = nx.clustering(G)
# Global Average clustering
print(f'Average clustering {nx.average_clustering(G)}')

## Central nodes

In [ ]:
# El método betweenness_centrality() nos regresa un diccionario con el betweenness de cada nodo
bet = nx.betweenness_centrality(G)
# Ordenamos los items del diccionario por una función lambda que toma el segundo elemento
# Items me regresa tuples 2 con llave y valor, nos sirve para iterar
# Podría ser lambda x: x[1], reverse=True
max_bet = sorted(bet.items(), key=lambda x: -x[1])
bet[max_bet[0][0]]
max_bet[0]

## Closenness

In [ ]:
# Lo mismo pero ahora para el closeness_centrality
# Qué tan rápido podemos llegar de un nodo a todos los demás
clos = nx.closeness_centrality(G)
max_clos = sorted(clos.items(), key=lambda x: -x[1])
clos[max_clos[0][0]]
max_clos[0][1]

## $k$-core decomposition

In [ ]:
# Para nx usamos core_number y nos regresa un diccionario con personajes y el core number de c/u
core = nx.core_number(G)
max_k = max(core.values())
# List comprehension
[x for x in core if core[x]==max_k][1:10] # Nos quedamos con los nodos que están en el main-core

In [ ]:
max_k

## Drawing

In [ ]:
# nx.draw_networkx(G)

## Guardamos la red para llevarla a graph-tool

In [ ]:
nx.write_graphml(G,pwd + "/red_interna_dir.xml")

In [ ]:
# g.properties 
# labels = g.vertex_properties['_graphml_vertex_id']
# nodes = {labels[x]: x for x in g.iter_vertices()}
# gt.graph_draw(g)
# largest_cc = gt.label_largest_component(g)
# p = gt.GraphView(g, vfilt=largest_cc)
# gt.graph_draw(p)

In [ ]:
cowsay.cow(
    ' Red interna del Colmex dirigida (follows).\n '
    + str(len(G)) + ' Número de nodes/usuarios.\n '
    + str(len(G.edges)) + ' Número de edges/follows.\n '
    + str(round(nx.attribute_assortativity_coefficient(G, 'status'),3)) + ' assortativity coefficient por status\n '
    + str(round(nx.attribute_assortativity_coefficient(G, 'center'),3)) + ' assortativity coefficient por centro\n '
    + str(round(nx.average_clustering(G),3)) + ' average clustering de la red interna Colmex\n '
    + str(max_bet[0][0]) + ' Usuario con mayor Betweenness: ' +str(round(max_bet[0][1],4)) + '\n'
    + str(max_clos[0][0]) + ' Usuario con mayor Closenness: ' +str(round(max_clos[0][1],4)) + '\n'
    + str(max_k) + ' Máximo k-core ' + '\n'
          )